In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

# !conda install -y -c conda-forge scikit-learn 
# !conda install -y plotly

# -y -c = yes channel, conda-forge = channel with the latest libraries

Our dataset:

In [2]:
data = pd.read_csv('data_insurance.csv', sep = ';', 
                   names = ['age', 'sex', 'bmi', 'children', 'smoker', 'canton', 'pbf', 'charges'])
data = data.iloc[1:]

data['charges'] = data['charges'].str.replace('’', '').apply(pd.to_numeric)
data['pbf'] = data['pbf'].apply(pd.to_numeric)
data['bmi'] = data['bmi'].apply(pd.to_numeric)
data['age'] = data['age'].apply(pd.to_numeric)
data['children'] = data['children'].apply(pd.to_numeric)

# Encode categorical variables.
data['female'] = data['sex'] == 'w'
data['male'] = data['sex'] == 'm'
data['smoker'] = data['smoker'].apply(lambda x: x=='ja')

# Encode cantons as a categorical variable.
for canton_name in data['canton'].unique():
    data[canton_name.lower()] = data['canton'] == canton_name

# Remove encoded categorical variables.
data = data.drop('sex', axis=1)
data = data.drop('canton', axis=1)

# 1 record with negative percentage of body fat.
data = data[data['pbf']>0]

# removing clients under 21 with 5 children
data = data[~((data['age'] <= 21) & (data['children'] == 5))]

# Divide the charges into categories. 
# ASK# Is there a more objective way to determine the categories? K-clustering? Derivatives?
bins = [0, 10000, 39000, 100000]
bin_labels = [0, 1, 2]
data['charges_level'] = pd.cut(data['charges'], bins=bins, labels=bin_labels, include_lowest=True, right=False)

data.head()

,age,bmi,children,smoker,pbf,charges,female,male,so,ag,bs,bl,charges_level
1,19,24.720,0,True,35.94,4253,True,False,True,False,False,False,0
2,18,29.416,1,False,26.86,2494,False,True,False,True,False,False,0
3,28,28.800,3,False,26.92,3138,False,True,False,True,False,False,0
4,33,20.564,0,False,7.98,1553,False,True,False,False,True,False,0
5,32,25.504,0,False,21.84,2768,False,True,False,False,True,False,0


In [16]:
data.describe()

,age,bmi,children,charges
count,1334.000000,1334.000000,1334.000000,1334.000000
mean,39.251874,26.935298,1.086957,6702.321589
std,14.040328,4.877313,1.192598,5977.900466
min,18.000000,15.168000,0.000000,1171.000000
25%,27.000000,23.437000,0.000000,3293.500000
50%,39.000000,26.720000,1.000000,4861.000000
75%,51.000000,30.155000,2.000000,7871.500000
max,64.000000,44.904000,5.000000,59703.000000


In [5]:
data.head()

,age,bmi,children,smoker,pbf,charges,female,male,so,ag,bs,bl,charges_level
1,19,24.720,0,True,35.94,4253,True,False,True,False,False,False,0
2,18,29.416,1,False,26.86,2494,False,True,False,True,False,False,0
3,28,28.800,3,False,26.92,3138,False,True,False,True,False,False,0
4,33,20.564,0,False,7.98,1553,False,True,False,False,True,False,0
5,32,25.504,0,False,21.84,2768,False,True,False,False,True,False,0


Our dataset is left-skewed and has many outliers, so using random forest might prove useful, since it is not that sensitive to skewed distributions.  
We have also assessed the importance of our variables with pca, so we can limit the number of predictors to 5 variables.

In [6]:
# Dropping the variables we won't use.
data = data.drop(['pbf', 'male', 'female'], axis=1)

In [7]:
from sklearn.model_selection import train_test_split

labels = np.array(data['charges_level'])
# The data we want to split = 'input', we drop dependent variables / labels which we want to predict.
input = data.drop('charges', axis=1).drop('charges_level', axis=1)

# The argument for the split function has to be an array.
input_list = list(input.columns)
input = np.array(input)

# Split the data into training and test sets.
# To make the tests reproducible, random_state gives the same output for each call.
train_input, test_input, train_labels, test_labels = \
    train_test_split(input, labels, test_size = 0.25, random_state = 42) 

# Shape of the dataset (to check if the input has the same no. of lines as the labels).
print('Training input shape:', train_input.shape) # 2D array
print('Training labels shape:', train_labels.shape) # 1D array
print('Testing input shape:', test_input.shape)
print('Testing labels shape:', test_labels.shape)

Training input shape: (1000, 8)
Training labels shape: (1000,)
Testing input shape: (334, 8)
Testing labels shape: (334,)


In [8]:
class Node:
    def __init__(self, depth, probabilities):
        self.depth = depth
        self.probabilities = probabilities
        
        self.left = None
        self.right = None
        
        self.column = None
        self.value = None
        
        # If this is a leaf node (no further splitting)
        self.is_terminal = False
        
class DecisionTree(object):
    def __init__(self, max_depth = 10, min_samples_leaf = 1, min_samples_split = 2):
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.min_samples_split = min_samples_split
        
        self.root = None
        
    def fit(self, X, labels):
        self.classes = np.unique(labels)
        self.root = Node(depth=0, probabilities=self.class_probabilities(labels))
        self.build_tree(X, labels, self.root)
        
    def class_probabilities(self, labels):
        probabilities = []
        n_all_samples = len(labels)
        
        for one_class in self.classes:
            n_class_samples = len(labels[labels == one_class])
            probabilities.append(n_class_samples / n_all_samples)
            
        return np.asarray(probabilities)
        
    def build_tree(self, X, labels, node):
        # Verify tree building constraints.
        if node.depth >= self.max_depth:
            node.is_terminal = True
            return
        if len(X) < self.min_samples_split:
            node.is_terminal = True
            return
        if len(np.unique(labels)) == 1:
            node.is_terminal = True
            return
        
        # Find the best possible split in this node.
        split_column, split_value = self.find_split(X, labels)
        
        # Check if split valid.
        if split_column is None:
            node.is_terminal = True
            return
        
        x_col = X[:, split_column]
        
        # Select inputs and labels for the left subtree.
        x_left = X[x_col <= split_value, :]
        labels_left = labels[x_col <= split_value]
        if len(x_left) < self.min_samples_leaf:
            node.is_terminal = True
            return
        
        # Select inputs and labels for the right subtree.
        x_right = X[x_col > split_value, :]
        labels_right = labels[x_col > split_value]
        if len(x_right) < self.min_samples_leaf:
            node.is_terminal = True
            return
        
        node.column = split_column
        node.value = split_value
        
        node.left = Node(depth=node.depth + 1, 
                         probabilities=self.class_probabilities(labels_left))
        node.right = Node(depth=node.depth + 1, 
                          probabilities=self.class_probabilities(labels_right))
        
        # Split the left and the right sub tree.
        self.build_tree(x_left, labels_left, node.left)
        self.build_tree(x_right, labels_right, node.right)
    
    def find_split(self, X, labels):
        best_column = None
        best_value = None
        best_info_gain = float('-inf')
        
        gini_current = 1 - np.sum(self.class_probabilities(labels)**2)
        
        # Try to split on each available column.
        for col in range(X.shape[1]):
            x_col = X[:, col]
            
            # Test the split on all column values.
            for x_value in x_col:
                
                # Split tree based on current column and x value.
                labels_left = labels[x_col <= x_value]
                # Split is only valid if there are values on both sides.
                if len(labels_left) == 0:
                    continue
                    
                # Split tree based on current column and x value.
                labels_right = labels[x_col > x_value]
                # Split is only valid if there are values on both sides.
                if len(labels_right) == 0:
                    continue
                
                # Calculate gini indexes for both subtrees.
                gini_left = 1 - np.sum(self.class_probabilities(labels_left)**2)
                gini_right = 1 - np.sum(self.class_probabilities(labels_right)**2)
                
                # Calculate information gain.
                info_gain = gini_current
                info_gain -= gini_left * len(labels_left) / len(labels)
                info_gain -= gini_right * len(labels_right) / len(labels)

                # Remember split if this is the best information gain so far.
                if info_gain > best_info_gain:
                    best_column = col
                    best_value = x_value
                    best_info_gain = info_gain
                    
        return best_column, best_value
    
    # Similar format to what sklearn outputs
    def print_tree(self, node=None):
        if node is None:
            node = self.root
            
        # Calculate the number of spaces to print a tree in console
        offset = ' ' * 3 * node.depth
    
        # There is no split in a terminal node, just print probabilities.
        if node.is_terminal:
            print(offset, node.probabilities)
            return

        print(offset, '|---', input_list[node.column], '<=', node.value)
        self.print_tree(node.left)

        print(offset, '|---', input_list[node.column], '>', node.value)
        self.print_tree(node.right)
        
    def predict(self, X):
        predictions = []
        
        # Find the leaf node for each test input.
        for x in X:
            node = self.root
            while not node.is_terminal:
                if x[node.column] > node.value:
                    node = node.right
                else:
                    node = node.left 
                    
            # Select the class that has the highest probability.
            predictions.append(np.argmax(node.probabilities))
        
        return np.asarray(predictions)

In [9]:
dt = DecisionTree(max_depth=4, min_samples_leaf = 5, min_samples_split = 10)
dt.fit(train_input, train_labels)

dt.print_tree()

 |--- smoker <= False
    |--- bmi <= 33.552
       |--- bmi <= 31.432
          |--- age <= 63
             [0.99541985 0.00458015 0.        ]
          |--- age > 63
             [0.75 0.25 0.  ]
       |--- bmi > 31.432
          |--- age <= 50
             [1. 0. 0.]
          |--- age > 50
             [0.39130435 0.60869565 0.        ]
    |--- bmi > 33.552
       |--- age <= 41
          |--- age <= 31
             [1. 0. 0.]
          |--- age > 31
             [0.69230769 0.30769231 0.        ]
       |--- age > 41
          |--- bmi <= 34.244
             [0.38461538 0.61538462 0.        ]
          |--- bmi > 34.244
             [0. 1. 0.]
 |--- smoker > False
    |--- bmi <= 26.72
       |--- age <= 53
          |--- age <= 46
             [0.98571429 0.01428571 0.        ]
          |--- age > 46
             [0.73333333 0.26666667 0.        ]
       |--- age > 53
          [0.15384615 0.84615385 0.        ]
    |--- bmi > 26.72
       |--- age <= 28
          |--- bmi <= 

In [10]:
from sklearn.metrics import accuracy_score
accuracy_score(test_labels, dt.predict(test_input))

0.9251497005988024

#### Sources
 
https://towardsdatascience.com/decision-tree-algorithm-in-python-from-scratch-8c43f0e40173    
https://www.youtube.com/watch?v=zboCGDMnU3I   
https://machinelearningmastery.com/implement-decision-tree-algorithm-scratch-python/   
https://www.analyticsvidhya.com/blog/2020/10/all-about-decision-tree-from-scratch-with-python-implementation/

